In [ ]:
%pip install 'vanna[chromadb,openai,postgres]'

In [ ]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
from openai import OpenAI
import os
from dotenv import load_dotenv

from contextlib import redirect_stdout, redirect_stderr
import io

## You will need to have the OpenAI key in the .env file, along with having it set to NEW_OPENAI_API_KEY=sk-....

In [2]:
# Load the .env file
load_dotenv()

# Retrieve the values using os.environ
api_key = os.getenv('NEW_OPENAI_API_KEY')

In [ ]:
print(f"The value of the api_key is the {api_key}")

## Running this code will generate a chroma.sqlite3 file, which is likely the ChromaDB?

The code below creates a chroma.sqlite3 file

In [3]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

# Running this code creates the chroma.sqlite3 file
vn = MyVanna(config={'api_key': api_key, 'model': 'gpt-3.5-turbo'})

In [ ]:
import requests
ip = requests.get('https://api.ipify.org').text
print(f"Your Colab runtime IP is: {ip}")

## Remember the host number might change! Change that whenever the VM stops running

A sample of what the database looks like is in the csv of this code or in /Vanna/courses.csv

In [4]:
vn.connect_to_postgres(host='35.226.19.55', dbname='ece_day_db', user='postgres', password='butlar', port='5432')

## Running this code will create the information schema (Directories full of training data and embeddings?)

In [ ]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.

# This SQL code retrieves all the information across all the tables in all tables within a database.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")


# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)

In [ ]:
vn.train(plan=plan)

In [ ]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

In [ ]:
vn.train(documentation="Student thesis lie in the team_name section of the datatable. So for example, Wiktor's project is a student thesis.")

In [ ]:
vn.train(documentation="My database outlines each group within Boston University senior Electrical and Computer Engineering (ECE) design course. The ECE Senior Design capstone course serves as an opportunity " \
"for students to execute the education they have gained in the" \
"classroom throughout their undergraduate careers, in order to" \
"produce prototypes for real-world clients. Student teams serve" \
"volunteer customers drawn from industry, government, small" \
"businesses, non-profits, schools, artists, faculty, and staff. ")

In [5]:
vn.train(documentation="For the team members, if I ask about a specific person, you should be able to prompt me with all of the other members. The team members are all part of the same project." \
"The members of one project are all shared. The only individuals that have their own project are the student thesis students.")

Adding documentation....


'a4c3c0c9-f124-581b-84b4-50a9d93ef491-doc'

In [7]:
vn.train(documentation="When I ask about a specific location, such as 'where is the location of John Smith's group?' you should give me the team location associated with it")

Adding documentation....


'd9ac4434-2ce1-5aa7-a0a0-2d3a18ba21c2-doc'

In [6]:
vn.train(documentation="Associate each person with their project. It is possessive. So when I ask what is John Smith's project, you should parse through the abstract summary and give me their" \
"associated project.")

Adding documentation....


'50a15d98-006f-59cc-b913-cc214247e5e6-doc'

In [ ]:
# You can remove training data if there's obsolete/incorrect information.
vn.remove_training_data(id='66230b0b-b604-5bad-96fe-99bc906f0502-doc')
training_data

In [ ]:
# %%capture captured_output
my_question = "How many student thesis projects are there? Can you give me an example about one of them?"
response = vn.ask(question=my_question, allow_llm_to_see_data=True)

In [ ]:
capt_vanna_ans= captured_output.stdout
if "ERROR" not in capt_vanna_ans:
    lines = capt_vanna_ans.splitlines()

    SQL_query_commands = ["SELECT", "AND", "WHERE", "FROM", ";", "gpt", "SQL", "LLM", "`"]
    filtered_lines = [line for line in lines if not any(substring in line for substring in SQL_query_commands)]
    vanna_ans_tables = "\n".join(filtered_lines)

    print(vanna_ans_tables)
else:
    print("Bruh happened")

In [ ]:
client = OpenAI()
client.api_key = api_key

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant in interpreting data tables into complete sentences and an intelligible response."},
    {"role": "user", "content": f"Answer the question of: {my_question}, given this data table of{vanna_ans_tables}"}
  ]
)

print(completion.choices[0].message.content)


In [ ]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

def answer_course_question_new(question: str):
    vn = MyVanna(config={'api_key': api_key, 'model': 'gpt-3.5-turbo'})
    vn.connect_to_postgres(host='34.134.126.254', dbname='tutorialDB', user='postgres', password='butlar', port='5432')
    # training_data = vn.get_training_data()
    # print(training_data)

    dummy_output = io.StringIO()
    dummy_error = io.StringIO()
    with redirect_stdout(dummy_output), redirect_stderr(dummy_error):
        response = vn.ask(question=question, allow_llm_to_see_data=True, print_results=False)
    
    if response[1]:
        print(f"The response is {response[1]}")
    return response[1]

newple = answer_course_question_new("What courses does Tali Moreshet teach?")

In [ ]:
print(newple[1])

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()